# Music Retrieval / Audio Identification

**Course:** Introduction to Information Retrieval (194.166)  
**Milestone 2.1:** Audio Processing  
**Group:** 26

| Student Name           | Student ID |
|------------------------|------------|
| Nikita Lysenko         | 12127033   |
| Yagmur Coban           | 12313405   |
| Nikolaus Winkelhofer  | 12317833   |

## Overview
Exercise 2 deals with a specific music retrieval task, namely audio identification. Milestone 2.1 covers audio processing aspects and measuring retrieval effectiveness under various query preprocessing and distortion conditions.

## 1. Obtaining Data and Query Preparation (30%)

- Original: the unmodified 10 sec extracted segment
- Noise: add noticeable (Gaussian) noise to the segment
- Coding: a highly compressed representation with clearly audible artifacts
- Mobile: play back the segments via a laptop or phone in an urban outdoor setting and record it with another device (phone). There is no need for the recording to be exactly 10 seconds, the recorded music segment can be truncated and/or surrounded by noise

## 1.1 Add Noise
used noise level 0.04.

## 1.2 Compress Track
compressed them to 16kbps

## 1.3 Play on other device and record it with microphone

# Task 2: Database Preparation (20%)

1.  **Spectrogram Generation**: Converts audio signals into the time-frequency domain using STFT.
2.  **Constellation Map Extraction**: Identifies local maxima (peaks) to create robust fingerprints.
3.  **Matching**: Compares query fingerprints against a database.
4.  **Evaluation**: Calculates detailed metrics including **Precision, Recall, F1-Score**, and **Accuracy** across different query categories (Original, Noise, Mobile, Coding).


#

In [1]:
import os, sys
import numpy as np
from numba import jit
import librosa
from scipy import ndimage
from matplotlib import pyplot as plt
import IPython.display as ipd
import time
import glob
import pickle
import pandas as pd
from tqdm.notebook import tqdm
import warnings
warnings.filterwarnings('ignore')

## Paths

In [2]:
BASE_PATH = "C:/5Semester/IR/Ex2"
DB_AUDIO_PATH = os.path.join(BASE_PATH, "raw_30s_audio-26", "26")
QUERY_AUDIO_PATH = os.path.join(BASE_PATH, "music_queries")
OUTPUT_DB_FILE = os.path.join(BASE_PATH, "db_fingerprints.pkl")
OUTPUT_CSV_FILE = os.path.join(BASE_PATH, "detailed_results.csv")

## Parameters

In [3]:
PARAMETER_SETS = [
    {'id': 0, 'dist_freq': 3, 'dist_time': 3},
    # {'id': 1, 'dist_freq': 4, 'dist_time': 2},
    # {'id': 2, 'dist_freq': 6, 'dist_time': 4},
    # {'id': 3, 'dist_freq': 9,  'dist_time': 5},
    # {'id': 4, 'dist_freq': 12, 'dist_time': 6},
    # {'id': 5, 'dist_freq': 15, 'dist_time': 8},
    # {'id': 6, 'dist_freq': 25, 'dist_time': 10},
    # {'id': 7, 'dist_freq': 35, 'dist_time': 15},
    # {'id': 8, 'dist_freq': 50, 'dist_time': 20},
    # {'id': 9,  'dist_freq': 10, 'dist_time': 5},
    # {'id': 10, 'dist_freq': 10, 'dist_time': 5},
    # {'id': 11, 'dist_freq': 7,  'dist_time': 30},
]

## Optimization

In [4]:
# Optimization: Limit the search for match start positions to the first 21 seconds
MAX_SCAN_START_SEC = 21.0

## 2.1 Compute Spectrogram
Loads audio and computes the Short-Time Fourier Transform (STFT).
We used the method of the reference jupyter notebook and added a max processing limit of 30s as stated in the task description.

In [5]:
def compute_spectrogram(fn_wav, Fs=22050, N=2048, H=1024, bin_max=None, frame_max=None):
    """Computes the magnitude spectrogram of an audio file.

    Args:
        fn_wav (str): Path to the audio file.
        Fs (int): Sampling rate in Hz.
        N (int): Window size in samples.
        H (int): Hop size in samples.
        bin_max (int): Maximum frequency bin index to return.
        frame_max (int): Maximum time frame index to return.
        duration (float): Duration in seconds to be loaded.

    Returns:
        Y (np.ndarray): Magnitude spectrogram.
    """

    x, Fs = librosa.load(fn_wav, sr=Fs, duration=30.0)
    X = librosa.stft(x, n_fft=N, hop_length=H, win_length=N, window='hann')
    if bin_max is None:
        bin_max = X.shape[0]
    if frame_max is None:
        frame_max = X.shape[0]
    Y = np.abs(X[:bin_max, :frame_max])
    return Y

## 2.2 Compute Constellation Map
Uses a maximum filter to find local peaks.

In [6]:
def compute_constellation_map(Y, dist_freq=7, dist_time=7, thresh=0.01):
    """Compute constellation map (implementation using image processing)

    Args:
        Y (np.ndarray): Spectrogram (magnitude)
        dist_freq (int): Neighborhood parameter for frequency direction (kappa) (Default value = 7)
        dist_time (int): Neighborhood parameter for time direction (tau) (Default value = 7)
        thresh (float): Threshold parameter for minimal peak magnitude (Default value = 0.01)

    Returns:
        Cmap (np.ndarray): Boolean mask for peak structure (same size as Y)
    """
    result = ndimage.maximum_filter(Y, size=[2 * dist_freq + 1, 2 * dist_time + 1], mode='constant')
    Cmap = np.logical_and(Y == result, result > thresh)
    return Cmap

## 2.3 Compute Matching Function
Matches the query against a database entry.

**Optimization**: Unlike the standard implementation, this pre-calculates the query tolerance (dilation)
    for performance and returns only the metrics of the single best match, not the full
    matching curve. It also limits the search range based on MAX_SCAN_START_SEC.

In [7]:
@jit(nopython=True)
def fast_scan_matching(C_D, C_Q_expanded, search_range):
    """
    Scans the query over the database document.
    Args:
        C_D: Database binary matrix (Freq x Time)
        C_Q_expanded: Query binary matrix, ALREADY EXPANDED with tolerance (Freq x Time)
        search_range: How many time frames to shift/scan

    Returns:
        best_shift (int): The starting index of the best match
        best_TP (int): The True Positive count of the best match
        best_N_DB (int): The number of peaks in the database segment (for FN calc)
    """
    L = C_D.shape[1]
    N = C_Q_expanded.shape[1]

    best_TP = -1
    best_shift = 0
    best_N_DB = 0

    # Fast sliding window loop
    for m in range(search_range):
        current_TP = 0

        # Optimized 2D array iteration for Numba
        # This counts overlapping peaks (logical AND)
        for r in range(C_D.shape[0]):
            for c in range(N):
                if C_D[r, m+c] and C_Q_expanded[r, c]:
                    current_TP += 1

        # Check if this shift is the new best match
        if current_TP > best_TP:
            best_TP = current_TP
            best_shift = m

            # Count actual peaks in the corresponding Database section
            # (Used to calculate False Negatives later)
            count_db = 0
            for r in range(C_D.shape[0]):
                for c in range(N):
                    if C_D[r, m+c]:
                        count_db += 1
            best_N_DB = count_db

    return best_shift, best_TP, best_N_DB

## 2. Task 2: Database Preparation

This section processes all reference audio files. It creates constellation maps for each defined parameter set and saves them into a `pickle` file. If the database file already exists, this step is skipped to save time.

In [8]:
def run_task2():
    print("----| START TASK 2: Database Preparation |----")

    if os.path.exists(OUTPUT_DB_FILE):
        print(f"INFO: Database '{OUTPUT_DB_FILE}' already exists.")
        print("--> Loading existing database...")
        return

    extensions = ['*.mp3', '*.wav', '*.m4a']
    db_files = []
    for ext in extensions:
        db_files.extend(glob.glob(os.path.join(DB_AUDIO_PATH, ext)))

    if not db_files:
        print(f"ERROR: No audio files found in '{DB_AUDIO_PATH}'!")
        return

    print(f"Found: {len(db_files)} files. Starting processing...")

    db_fingerprints = {p['id']: {} for p in PARAMETER_SETS}


    for fpath in tqdm(db_files, desc="Indexing Database"):
        fname = os.path.basename(fpath)

        try:
            # Only the first 30 seconds are indexed
            Y = compute_spectrogram(fpath)

            for params in PARAMETER_SETS:
                pid = params['id']
                Cmap = compute_constellation_map(Y,
                                                 dist_freq=params['dist_freq'],
                                                 dist_time=params['dist_time'])
                db_fingerprints[pid][fname] = Cmap

        except Exception as e:
            tqdm.write(f"  Error with file {fname}: {e}")
            continue

    print(f"Saving database to '{OUTPUT_DB_FILE}'...")
    with open(OUTPUT_DB_FILE, 'wb') as f:
        pickle.dump(db_fingerprints, f)
    print("Task 2 completed successfully.")
run_task2()

----| START TASK 2: Database Preparation |----
INFO: Database 'C:/5Semester/IR/Ex2\db_fingerprints.pkl' already exists.
--> Loading existing database...


## 3. Task 3: Retrieval & Evaluation

In this step, we match the query files (Original, Noise, Mobile, Coding) against our database.

**Metrics Calculation per Query:**
* **TP (True Positives):** Number of matching peaks.
* **FP (False Positives):** Peaks present in the query but missing in the DB match.
* **FN (False Negatives):** Peaks present in the DB match but missing in the query.
* **Precision**: $TP / (TP + FP)$
* **Recall**: $TP / (TP + FN)$
* **F1-Score**: $2 \cdot \frac{Precision \cdot Recall}{Precision + Recall}$

The detailed results are saved to a CSV file.

In [9]:
def run_task3():
    print("----| START TASK 3: Optimized Retrieval |----")

    if not os.path.exists(OUTPUT_DB_FILE):
        print("Error: Database file not found. Run Task 2 first.")
        return pd.DataFrame()

    print("Loading database...")
    with open(OUTPUT_DB_FILE, 'rb') as f:
        db_fingerprints = pickle.load(f)

    # Find all query files
    search_pattern = os.path.join(QUERY_AUDIO_PATH, "**", "*.*")
    query_files = glob.glob(search_pattern, recursive=True)
    query_files = [f for f in query_files if f.lower().endswith(('.wav', '.mp3', '.m4a'))]

    detailed_records = []

    # OPTIMIZATION 1: Loop over Queries FIRST
    # This prevents reloading the slow MP3/WAV file 12 times
    for q_path in tqdm(query_files, desc="Processing Queries"):
        q_name = os.path.basename(q_path)

        # Determine Category
        q_type = 'Unknown'
        path_lower = q_path.lower()
        if 'original' in path_lower: q_type = 'Original'
        elif 'noise' in path_lower: q_type = 'Noise'
        elif 'coding' in path_lower: q_type = 'Coding'
        elif 'mobile' in path_lower: q_type = 'Mobile'
        if q_type == 'Unknown': continue

        try:
            # Load Audio ONCE per file
            Y_q = compute_spectrogram(q_path)

            # Loop over your 12 configurations
            for params in PARAMETER_SETS:
                pid = params['id']
                config_name = f"ID{pid}_df{params['dist_freq']}_dt{params['dist_time']}"

                # Compute Constellation Map
                C_q = compute_constellation_map(Y_q,
                                                dist_freq=params['dist_freq'],
                                                dist_time=params['dist_time'])

                M_query_sum = np.sum(C_q) # Total peaks in query (for FP calc)

                # OPTIMIZATION 2: Expand Query ONCE (Pre-calculation)
                # We apply the tolerance (dilation) here, before the DB loop
                tol_freq = 1
                tol_time = 1
                C_q_expanded = ndimage.maximum_filter(
                    C_q,
                    size=(2*tol_freq+1, 2*tol_time+1),
                    mode='constant'
                )

                # Prepare for matching
                current_db = db_fingerprints[pid]
                Fs = 22050
                H = 1024
                # Limit search to first 21 seconds (optional optimization)
                MAX_SCAN_START_SEC = 21.0
                max_frames = int(MAX_SCAN_START_SEC * Fs / H)

                best_match_name = None
                best_TP = -1
                best_N_DB = 0
                best_shift = 0

                start_match = time.time()

                # OPTIMIZATION 3: Numba Scan Loop
                for db_name, C_db in current_db.items():
                    L = C_db.shape[1]
                    N = C_q_expanded.shape[1]
                    M = L - N

                    if M < 0: continue # Query longer than DB track

                    # Determine search range
                    search_range = min(M + 1, max_frames)

                    # CALL THE NEW FAST FUNCTION
                    # Note: We pass C_q_expanded, not C_q
                    shift, tp, n_db = fast_scan_matching(C_db, C_q_expanded, search_range)

                    if tp > best_TP:
                        best_TP = tp
                        best_N_DB = n_db
                        best_shift = shift
                        best_match_name = db_name

                query_duration = time.time() - start_match

                # Calculate Metrics (Same as before)
                is_correct = False
                if best_match_name:
                    # Simple check: does query filename "12345" match DB filename "12345"?
                    # You may need to adjust this depending on your exact filenames
                    q_id = q_name.split('_')[0].split('.')[0]
                    db_id = best_match_name.split('_')[0].split('.')[0]
                    is_correct = (q_id in db_id) or (db_id in q_id)

                TP = best_TP
                FN = best_N_DB - TP
                FP = M_query_sum - TP

                # Avoid division by zero
                precision = TP / (TP + FP) if (TP + FP) > 0 else 0
                recall = TP / (TP + FN) if (TP + FN) > 0 else 0
                f1_score = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0

                detailed_records.append({
                    'Category': q_type,
                    'Query': q_name,
                    'Configuration': config_name,
                    'Matched DB File': best_match_name,
                    'TP': TP,
                    'FN': FN,
                    'FP': FP,
                    'Precision': round(precision, 4),
                    'Recall': round(recall, 4),
                    'F1 Score': round(f1_score, 4),
                    'Shift': int(best_shift),
                    'Query Time (s)': round(query_duration, 4),
                    'Correct': is_correct
                })
        except Exception as e:
            print(f"Error processing {q_name}: {e}")
            continue

    # Save Results
    df_results = pd.DataFrame(detailed_records)
    if not df_results.empty:
        df_results.to_csv(OUTPUT_CSV_FILE, index=False)
        print(f"Saved results to {OUTPUT_CSV_FILE}")

    return df_results

# Run it
df_results = run_task3()

----| START TASK 3: Optimized Retrieval |----
Loading database...


Processing Queries:   0%|          | 0/80 [00:00<?, ?it/s]

Saved results to C:/5Semester/IR/Ex2\detailed_results.csv


In [17]:
if not df_results.empty:
    # 1. Define Styling Function
    def highlight_correct(val):
        if val:
            return 'background-color: #198754; color: white; font-weight: bold;'
        else:
            return 'background-color: #dc3545; color: white; font-weight: bold;'

    # --- ÄNDERUNG START ---

    # 2. Create a Grouping Column (Base Name)
    # Wir erstellen eine Spalte, die alles nach dem letzten Punkt abschneidet
    # Aus "564334.wav" und "564334.mp3" wird beides "564334"
    df_results['Query_Group'] = df_results['Query'].astype(str).str.rsplit('.', n=1).str[0]

    # Get unique groups (sorted)
    unique_groups = sorted(df_results['Query_Group'].unique())

    print(f"--- Detailed Results by Query Group ({len(unique_groups)} Groups) ---")

    for group_name in unique_groups:
        # Filter dataframe for this specific group (Basis-Name)
        df_group = df_results[df_results['Query_Group'] == group_name].copy()

        # Die Hilfsspalte 'Query_Group' entfernen wir für die Anzeige wieder,
        # damit die Tabelle sauber bleibt (oder du lässt sie drin, wenn du willst)
        df_display = df_group.drop(columns=['Query_Group'])

        # Create a text header using the group name (e.g., "564334")
        print(f"\nQuery Group: {group_name}")

        # Apply Styling on the group dataframe
        styled_df = df_display.style.applymap(highlight_correct, subset=['Correct'])
        styled_df = styled_df.format({
            'Precision': '{:.4f}',
            'Recall': '{:.4f}',
            'F1 Score': '{:.4f}',
            'Query Time (s)': '{:.2f}'
        })

        # Display the table (jetzt sind wav und mp3 zusammen)
        display(styled_df)

    # --- ÄNDERUNG ENDE ---

    # 3. Display Summary Pivot Table (At the end)
    print("\n" + "="*40)
    print("Summary: Accuracy (%)")
    print("="*40)

    pivot_acc = df_results.pivot_table(
        index='Configuration',
        columns='Category',
        values='Correct',
        aggfunc='mean'
    ) * 100

    display(pivot_acc.style.background_gradient(cmap='RdYlGn', vmin=0, vmax=100).format("{:.2f}"))

else:
    print("No results generated.")

--- Detailed Results by Query Group (20 Groups) ---

Query Group: 566726


,Category,Query,Configuration,Matched DB File,TP,FN,FP,Precision,Recall,F1 Score,Shift,Query Time (s),Correct,Query_No_Ext
0,Coding,566726.mp3,ID0_df3_dt3,566726.mp3,457,2886,335,0.5770,0.1367,0.2210,0,5.63,True,566726
20,Mobile,566726.m4a,ID0_df3_dt3,566726.mp3,888,2834,2833,0.2386,0.2386,0.2386,377,5.54,True,566726
40,Noise,566726.wav,ID0_df3_dt3,566726.mp3,2159,1180,1427,0.6021,0.6466,0.6235,0,5.49,True,566726
60,Original,566726.wav,ID0_df3_dt3,566726.mp3,3337,6,21,0.9937,0.9982,0.9960,1,5.50,True,566726



Query Group: 568126


,Category,Query,Configuration,Matched DB File,TP,FN,FP,Precision,Recall,F1 Score,Shift,Query Time (s),Correct,Query_No_Ext
1,Coding,568126.mp3,ID0_df3_dt3,568126.mp3,348,1718,364,0.4888,0.1684,0.2505,0,5.61,True,568126
21,Mobile,568126.m4a,ID0_df3_dt3,124826.mp3,1223,6902,4005,0.2339,0.1505,0.1832,138,5.73,False,568126
41,Noise,568126.wav,ID0_df3_dt3,568126.mp3,1258,803,2283,0.3553,0.6104,0.4491,0,5.67,True,568126
61,Original,568126.wav,ID0_df3_dt3,568126.mp3,2046,15,13,0.9937,0.9927,0.9932,0,5.54,True,568126



Query Group: 575226


...

,Category,Query,Configuration,Matched DB File,TP,FN,FP,Precision,Recall,F1 Score,Shift,Query Time (s),Correct,Query_No_Ext
18,Coding,783626.mp3,ID0_df3_dt3,783626.mp3,337,1564,277,0.5489,0.1773,0.2680,285,5.88,True,783626
38,Mobile,783626.m4a,ID0_df3_dt3,124826.mp3,896,4574,2805,0.2421,0.1638,0.1954,371,4.92,False,783626
58,Noise,783626.wav,ID0_df3_dt3,124826.mp3,929,4305,3013,0.2357,0.1775,0.2025,72,5.73,False,783626
78,Original,783626.wav,ID0_df3_dt3,783626.mp3,1764,129,295,0.8567,0.9319,0.8927,287,5.64,True,783626



Query Group: 783926


,Category,Query,Configuration,Matched DB File,TP,FN,FP,Precision,Recall,F1 Score,Shift,Query Time (s),Correct,Query_No_Ext
19,Coding,783926.mp3,ID0_df3_dt3,783926.mp3,221,824,171,0.5638,0.2115,0.3076,233,5.97,True,783926
39,Mobile,783926.m4a,ID0_df3_dt3,124826.mp3,1063,5307,3377,0.2394,0.1669,0.1967,180,5.90,False,783926
59,Noise,783926.wav,ID0_df3_dt3,124826.mp3,962,4273,3085,0.2377,0.1838,0.2073,70,5.52,False,783926
79,Original,783926.wav,ID0_df3_dt3,783926.mp3,926,101,131,0.8761,0.9017,0.8887,237,5.41,True,783926



Summary: Accuracy (%)


Category,Coding,Mobile,Noise,Original
Configuration,,,,
ID0_df3_dt3,100.00,40.00,75.00,100.00


## 4. Analysis & Results

Below are the aggregated results of the experiment.
1.  **Detailed Table (Excerpt):** Shows metrics per specific query.
2.  **Accuracy Table:** Percentage of correctly identified songs per category and parameter set.
3.  **F1-Score Table:** Average F1-Score per category.